In [1]:
// #load "NbHelper.fsx"

#I "../Vide.DSP/bin/Debug/netstandard2.0"
#r "Vide.Common.dll"
#r "Vide.DSP.dll"

#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"

Installed Packages Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

Loading extensions from `C:\Users\ronal\.nuget\packages\plotly.net.interactive\4.2.0\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

In [43]:
open Vide
open Vide.DSP
open Plotly.NET
open Plotly.NET.LayoutObjects

let showLine (values: _ list) =
    let xvalues = List.init (values |> List.length) float
    Chart.Line(xvalues, values) |> Chart.withSize(1200.0, 500.0) |> Chart.withMargin(Margin.init(5,5,15,5))


In [44]:
showLine [ 0. .. 10. ]


<!-- Plotly chart will be drawn inside this DIV -->